In [2]:
from datasets import load_dataset 

ds1 = load_dataset("gaotang/filtered_sky_code_8k_math_10k_rubric_reasoning_test", split="train")
ds2 = load_dataset("gaotang/filtered_sky_code_8k_math_10k_rubric_evidence_classify_weight_rest_0420_shuffle_test", split="train")

In [3]:
ds1[0]

{'context_messages': [{'content': 'Please act as an impartial judge and evaluate the quality of the responses provided by two AI Chatbots to the Client question displayed below. \n\n[Client Question]\nIdentify the data structure used to represent a specific tree and implement a method to traverse this tree both Breadth first and Depth first. A Tree with 8 nodes including A as the root node, connected to B, C, D, E, F, G, and H as child nodes, demonstrating the hierarchical nature. Depth first traversal should consider left-most children first.\n\n[The Start of Chatbot A\'s Response]\nIn this case, we can use the adjacency list method to represent the tree. Here is a Python program that performs both Breadth-first and Depth-first traversal:\n\n```python\nfrom collections import deque\n\n\ntree = {\n    \'A\' : [\'B\', \'C\', \'D\', \'E\', \'F\', \'G\', \'H\'],\n    \'B\' : [],\n    \'C\' : [],\n    \'D\' : [],\n    \'E\' : [],\n    \'F\' : [],\n    \'G\' : [],\n    \'H\' : []\n}\n\ndef 

In [ ]:
ds2[0]

{'context_messages': [{'content': "Please act as an impartial judge and evaluate the quality of the responses provided by two AI Chatbots to the Client's question displayed below.\n\nFirst, classify the task into one of two categories: <type>Reasoning</type> or <type>Chat</type>.\n- Use <type>Reasoning</type> for tasks that involve math, coding, or require domain knowledge, multi-step inference, logical deduction, or combining information to reach a conclusion.\n- Use <type>Chat</type> for tasks that involve open-ended or factual conversation, stylistic rewrites, safety questions, or general helpfulness requests without deep reasoning.\n\nIf the task is Reasoning:\n1. Solve the Client's question yourself and present your final answer within <solution>...</solution> tags.\n2. Evaluate the two Chatbot responses based on correctness, completeness, and reasoning quality, referencing your own solution.\n3. Include your evaluation inside <eval>...</eval> tags, quoting or summarizing the Chat

In [6]:
len(ds2[0]['context_messages'][0]['content'])

2871

In [7]:
import time
from contextlib import contextmanager

@contextmanager
def timer(label: str):
    """Print wall‑clock run‑time of the wrapped block."""
    t0 = time.perf_counter()          # high‑resolution clock
    yield
    dt = time.perf_counter() - t0
    print(f"{label:<25} : {dt:8.4f} s")

with timer("Hey"):
    print("3")

3
Hey                       :   0.0000 s


In [8]:
# fast_format.py  (final)
import re
from functools import lru_cache

# ---------- tiny helpers ----------
_WS   = r'\s*'                  # optional whitespace
_TEXT = r'.*?\S.*?'             # at least one non‑space char somewhere inside

# ---------- pre‑compiled pieces ----------
CHAT_TYPE_RE   = re.compile(r'<type>' + _WS + r'Chat'      + _WS + r'</type>', re.I)
REASON_TYPE_RE = re.compile(r'<type>' + _WS + r'Reasoning' + _WS + r'</type>', re.I)

RUBRIC_RE      = re.compile(r'<rubric>.*?<justify>' + _TEXT + r'</justify>.*?</rubric>',
                            re.I | re.S)
SOLUTION_RE    = re.compile(r'<solution>' + _TEXT + r'</solution>', re.I | re.S)

# (<quote_A>|<summary_A>) and (<quote_B>|<summary_B>) in *either* order
EVAL_RE = re.compile(
    r'<eval>.*?(?:'
    r'(?:(?:<quote_A>|<summary_A>).+?(?:<quote_B>|<summary_B>))'
    r'|'
    r'(?:(?:<quote_B>|<summary_B>).+?(?:<quote_A>|<summary_A>))'
    r')'
    r'.*?</eval>',
    re.I | re.S
)

ANSWER_RE = re.compile(
    r'<answer>' + _WS + r'(?:\S.*?\S|\S)' + _WS + r'</answer>',
    re.I | re.S
)

# ---------- main predicate ----------
@lru_cache(maxsize=4096)
def check_task_format(text: str) -> bool:
    """
    Return True if *text* is a valid Chat‑ or Reasoning‑task snippet,
    otherwise False.  All regexes are linear‑time and compiled once.
    """
    if CHAT_TYPE_RE.search(text):
        return (RUBRIC_RE.search(text) and
                EVAL_RE.search(text)   and
                ANSWER_RE.search(text))

    if REASON_TYPE_RE.search(text):
        return (SOLUTION_RE.search(text) and
                EVAL_RE.search(text)    and
                ANSWER_RE.search(text))

    return False


def format_reward(s: str) -> float:
    """Return 1.0 for a well‑formed task, else 0.0 (cheap wrapper)."""
    return 1.0 if check_task_format(s) else 0.0


In [9]:
if __name__ == "__main__":
    sample_chat = """
    <type>Chat</type>
    <rubric><justify>something</justify></rubric>
    <eval><quote_B>x</quote_B><quote_A>y</quote_A></eval>
    <answer> z </answer>
    """
    sample_reason = """
    <type>Reasoning</type>
    <solution>foo</solution>
    <eval><summary_A>α</summary_A> ... <quote_B>β</quote_B></eval>
    <answer>Y</answer>
    """
    for t in (sample_chat, sample_reason, "<type>Other</type>"):
        print(check_task_format(t))
# → True, True, False


<re.Match object; span=(135, 155), match='<answer> z </answer>'>
<re.Match object; span=(128, 146), match='<answer>Y</answer>'>
False
